In [ ]:
import random
import math

import matplotlib.pyplot as plt
import numpy as np
import deepcopy

In [67]:
GROUP_ID = "04"
ALG = "QLearning"
TRACK_NAME = "tracks/2-track.txt"
CRASH_POS = "RST"

In [68]:
def bresenhamsAlgorithm(x1, x2, y1, y2):
    dx = math.fabs(x1 -x2)
    dy = math.fabs(y1-y2)
    sx = (-1 if x2-x1 <0 else 1)
    sy = (-1 if y2-y1 <0 else 1)
    err = dx-dy
    p = 2* (dx-dy)
    linePoints = []
    while True:
        linePoints.append((x1, y1))
        if(x1 == x2 and y1 == y2):
            break
        e2 = 2 * err
        if e2 > -dy:
            err = err -dy
            x1 = x1 +sx
        if e2 < dx:
            err = err +dx
            y1 = y1 + sy

    return linePoints


In [69]:
print(bresenhamsAlgorithm(2,3,2,3))

[(2, 2), (3, 3)]


In [70]:
def makePlot(track):
    rows = track.size[1]
    cols = track.size[0]

    coloring = np.zeros((rows,cols,3))
    for y in range(rows):
        for x in range(cols):
            if track.array[x][y] == '.':
                coloring[y,x] = [1,1,1]
            elif track.array[x][y] == 'S':
                coloring[y,x] = [0,1,0]
            elif track.array[x][y] == 'F':
                coloring[y,x] = [1,0,0]
            else:
                coloring[y,x] = [0,0,0]

    plt.axis('off')
    plt.imshow(coloring)
    return plt


In [71]:
class RaceTrack:
    def __init__(self, file):
        with(open(file , "r") as f):
            lines = f.readlines()
            dims = lines[0]
            self.size = [0,0]
            self.size[0] = int(dims.split(",")[1].strip("\n"))
            self.size[1] = int(dims.split(",")[0])
            self.array = [[_ for y in range(self.size[1])] for x in range(self.size[0])]
            self.startStates = []
            self.finishStates = []
            self.walls = []
            self.track = []
            yIndex = 0
            for line in lines[1:]:
                xIndex = 0
                for char in line[:-1]:
                    if(char == "S"):
                        self.startStates.append((xIndex, yIndex))
                    elif(char == "F"):
                        self.finishStates.append((xIndex, yIndex))
                    elif(char == "#"):
                        self.walls.append((xIndex, yIndex))
                    else:
                        self.track.append((xIndex, yIndex))
                    self.array[xIndex][yIndex] = char
                    xIndex += 1
                yIndex += 1

    #prints track with car to command line, used for testing
    def printTrack(self, car):
        print("\n")
        for y in range(self.size[1]):
            line = ''
            for x in range(self.size[0]):
                if(car.xPos == x and car.yPos == y):
                    line += "C"
                else:
                    line += self.array[x][y]
            print(line)


In [ ]:
class Car:
    def __init__(self, racetrack):
        self.startState = random.choice(racetrack.startStates)
        self.xPos = self.startState[0]
        self.yPos = self.startState[1]
        self.xVel = 0
        self.yVel = 0
        self.xAccel = 0
        self.yAccel = 0
        self.track = racetrack
        self.timeSteps = 0

    def updatePos(self, xAccel, yAccel, resetAfter):
        iXpos = self.xPos
        iYpos = self.yPos
        iXvel = self.xVel
        iYvel = self.yVel
        iXAcc = self.xAccel
        iYAcc = self.yAccel
        #print("xPos:", self.xPos, "yPos:", self.yPos, "\nxVel:", self.xVel, "yVel:", self.yVel, "\nxAccel:", xAccel, "yAccel:", yAccel )
        #accounting for oil slicks in the track
        randomNum = random.uniform(0,1)
        if(randomNum <= 0.2):
            oilSlick = True
        else:
            oilSlick = False
        if(oilSlick):
            self.xAccel = 0
            self.yAccel = 0
        else:
            self.xAccel = xAccel
            self.yAccel = yAccel
        self.xVel += self.xAccel
        self.yVel += self.yAccel
        if(self.xVel > 5):
            self.xVel = 5
        if(self.xVel < -5):
            self.xVel = -5
        if(self.yVel > 5):
            self.yVel = 5
        if(self.yVel < -5):
            self.yVel = -5
        self.xPos += self.xVel
        self.yPos += self.yVel
        #checking if the car collided with a wall
        carCourse = bresenhamsAlgorithm(iXpos, self.xPos, iYpos, self.yPos)
        crashed = False
        finished = False
        for i in range(len(carCourse)):
            pos = carCourse[i]
            #crash
            if(pos in self.track.walls):
                crashed = True
                nearestPoint = carCourse[i-1]
                break
            if(pos in self.track.finishStates):
                finished = True
                finishPoint = pos
        if(crashed):
            self.resetCar()
            if(CRASH_POS == "NRST"):
                self.xPos = nearestPoint[0]
                self.yPos = nearestPoint[1]
            # else:
            #     # print("Fully reset")
        if(finished):
            self.resetCar()
            self.xPos = finishPoint[0]
            self.yPos = finishPoint[1]
            # print("Finished the race!")
        if(resetAfter):
            self.xPos = iXpos
            self.yPos = iYpos
            self.xVel = iXvel
            self.yVel = iYvel
            self.xAccel = iXAcc
            self.yAccel = iYAcc


    def resetCar(self):
        self.xVel = 0
        self.yVel = 0
        self.xAccel = 0
        self.yAccel = 0
        self.xPos = self.startState[0]
        self.yPos = self.startState[1]
        #print("xPos:", self.xPos, "yPos:", self.yPos, "\nxVel:", self.xVel, "yVel:", self.yVel, "\nxAccel:", self.xAccel, "yAccel:", self.yAccel)


In [ ]:
def qLearning(learningRate, discountFactor, epochs, explorationProb, racetrack):
    car = Car(racetrack)
    possibleStates = racetrack.startStates + racetrack.track + racetrack.finishStates
    possibleActions = []
    startState = car.startState
    for xA in [-1,0,1]:
        for yA in [-1,0,1]:
            possibleActions.append((xA, yA))
    qTable = np.zeros((len(possibleStates), len(possibleActions)))
    counterArr = []
    epochArr = []
    name = "qLearning_L=" + str(learningRate) + "_D="+ str(discountFactor) + "_E=" + str(explorationProb) + "_epochs=" + str(epochs) + ".png"
    for epoch in range(epochs):
        if(explorationProb >= 0.01):
            explorationProb *= 0.8
        if(learningRate >= 0.01):
            learningRate *= 0.999
        print(explorationProb , learningRate)
        print(epoch)
        car.resetCar()
        currentState = possibleStates.index(startState)
        counter = 0
        while True:
            counter += 1
            if(random.uniform(0,1) < explorationProb):
                actionChosenIndex = random.randint(0, len(possibleActions)-1)
            else:
                actionChosenIndex = np.argmax(qTable[currentState])
            actionChosen = possibleActions[actionChosenIndex]
            car.updatePos(actionChosen[0],actionChosen[1], False)
            carState = (car.xPos, car.yPos)
            nextState = possibleStates.index(carState)
            if(carState in racetrack.finishStates):
                reward = 0
            else:
                reward = -1
            qTable[currentState][actionChosenIndex] += learningRate * (reward + discountFactor * np.max(qTable[nextState]) - qTable[currentState][actionChosenIndex])
            currentState = nextState
            if(carState in racetrack.finishStates):
                break
        #print("In", counter, "steps")
        counterArr.append(counter)
        epochArr.append(epoch)
    plt.plot(epochArr, counterArr)
    plt.xlabel("Epochs")
    plt.ylabel("Steps")
    plt.savefig(name)
    plt.show()


    #running the actual agent
    car.resetCar()
    x_coord = [c.xPos]
    y_coord = [c.yPos]
    currentState = possibleStates.index(startState)
    while True:
        actionChosenIndex = np.argmax(qTable[currentState])
        actionChosen = possibleActions[actionChosenIndex]
        car.updatePos(actionChosen[0],actionChosen[1], False)
        x_coord.append(car.xPos)
        y_coord.append(car.yPos)
        carState = (car.xPos, car.yPos)
        nextState = possibleStates.index(carState)
        currentState = nextState
        if(carState in racetrack.finishStates):
            break

    graph = makePlot(r)
    graph.plot(x_coord,y_coord,linestyle = '-', color = 'blue')
    graph.show()
    return qTable



r = RaceTrack(TRACK_NAME)
c = Car(r)
x_coord = [c.xPos]
y_coord = [c.yPos]
#graph = makePlot(r)
#r.printTrack(c)
qTable = qLearning(0.5,0.99, 5000, 0.8, r)


0.6400000000000001 0.4995
0


In [ ]:
def act(oldY,oldX,oldVY,oldVX,accel,racetrack):
    randomNum = random.uniform(0,1)
    if(randomNum <= 0.2):
        oilSlick = True
    else:
        oilSlick = False
    if(oilSlick):
        accel = (0,0)
    newVX = oldVX + accel[0]
    newVY = oldVY + accel[1]
    if(newVX > 5):
        newVX = 5
    if(newVX < -5):
        newVX = -5
    if(newVY > 5):
        newVY = 5
    if(newVY < -5):
        newVY = -5
    newX = oldX + newVX
    newY = oldY + newVY
    #checking if the car collided with a wall
    carCourse = bresenhamsAlgorithm(oldX, newX, oldY, newY)
    crashed = False
    finished = False
    for i in range(len(carCourse)):
        pos = carCourse[i]
        #crash
        if(pos in racetrack.walls):
            crashed = True
            nearestPoint = carCourse[i-1]
            break
        if(pos in racetrack.finishStates):
            finished = True
            finishPoint = pos
    if(crashed):
        start = random.choice(racetrack.startStates)
        newVX = 0
        newVY = 0
        newX = start[0]
        newY = start[1]
        if(CRASH_POS == "NRST"):
            newX = nearestPoint[0]
            newY = nearestPoint[1]
        # else:
        #     # print("Fully reset")

    return newY, newX, newVY, newVX
    


def valueIteration(racetrack, error, discount):
    rows = racetrack.size[1]
    cols = racetrack.size[0]
    velRange = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
    actions = [(-1,-1), (0,-1), (1,-1),
           (-1,0) , (0,0),  (1,0),
           (-1,1) , (0,1),  (1,1)]

    values = [[[[random() for _ in velRange] for _ in velRange] for _ in range(cols)] 
              for _ in range(rows)]

    for y in range(rows):
        for x in range(cols):
            if racetrack.array[x][y] in racetrack.finishStates:
                for vy in velRange:
                    for vx in velRange:
                        values[y][x][vy][vx] = 0

    Q = [[[[[random() for _ in actions] for _ in velRange] for _ in (
        velRange)] for _ in range(cols)] for line in range(rows)]
    
    for y in range(rows):
        for x in range(cols):
            if racetrack.array[x][y] in racetrack.finishStates:
                for vy in velRange:
                    for vx in velRange:
                        for a, act in enumerate(actions):
                            Q[y][x][vy][vx][act] = 0

    notDone = True
    while(notDone):
        valuesPrev = deepcopy(values)

        delta = 0.0

        for y in range(rows):
            for x in range(cols):
                for vy in velRange:
                    for vx in velRange:

                        for ai,a in enumerate(actions):

                            if racetrack.array[x][y] in racetrack.finishStates:
                                r = 0
                            else:
                                r = -1

                            newY, newX, newVY, newVX = act(y,x,vy,vx,a,racetrack)

                            valNew = valuesPrev[newY][newX][newVY][newVX]

                            newY, newX, newVY, newVX = act(y,x,vy,vx,a,racetrack)

                            valFailedAct = valuesPrev[newY][newX][newVY][newVX]

                            expected = (0.80*valNew) + (0.20*valFailedAct)

                            Q[y][x][vy][vx][ai] = r + (discount * expected)

                        argMaxQ = np.argmax(Q[y][x][vy][vx])
                        values[y][x][vy][vx] = Q[y][x][vy][vx][argMaxQ]

        for y in range(rows):
            for x in range(cols):
                if racetrack.array[x][y] in racetrack.finishStates:
                    for vy in velRange:
                        for vx in velRange:
                            values[y][x][vy][vx] = 0

        if delta < error: 
            notDone = False
    
    pi = {}
    for y in range(rows):
        for x in range(cols):
            for vy in velRange:
                for vx in velRange:
                    pi[(y,x,vy,vx)] = actions[np.argmax(Q[y][x][vy][vx])]

    return(pi)



In [65]:
while True:
    userInput = input("Enter xAccel and yAccel: ")
    xA = int(userInput.split(",")[0]) + x_coord[len(x_coord)-1]
    yA = int(userInput.split(",")[1]) + y_coord[len(y_coord)-1]
    x_coord.append(xA)
    y_coord.append(yA)
    c.updatePos(xA, yA)
    graph = makePlot(r)
    graph.plot(x_coord,y_coord,linestyle = '-', color = 'blue')
    graph.show()
    r.printTrack(c)

KeyboardInterrupt: Interrupted by user